In [ ]:
from collections import Counter
import itertools
from itertools import zip_longest
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, show
import nltk
from nltk.corpus import stopwords
import numpy as np
from numpy.linalg import norm
import pandas as pd
import seaborn as sns
from scipy import sparse
from scipy.sparse import linalg 
from scipy.sparse import dok_matrix
from scipy.sparse import csr_array
from scipy.sparse import find
from scipy.sparse.linalg import svds
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

from pickleshare import *

db = PickleShareDB('~/.ipython/profile_default/db/autorestore')

dictionary = db['dictionary']

dictionary_lookup = db['dictionary_lookup']

sparse_word_doc_matrix = db['sparse_word_doc_matrix']